In [1]:
from yt_dlp import YoutubeDL
import json
import requests
from bs4 import BeautifulSoup
from scraperapi_key import api_key

# Scrape YouTube Search Results

In [ ]:
search_query = ['aliceinwonderland', 'aliceinwonderlandmovie', 'aliceinwonderland-liveaction', 'alice2010', 'alice in wonderland movie', 'aliceinwonderlandmoviereview','alicereview', 'alice in wonderland','timburton', 'miawasikowska', 'johnnydepp', 'helenabonhancarter']

url = f'https://www.youtube.com/results?search_query={search_query}'
params = {
    'api_key': api_key,
    'url': url,
    'render': 'true'
}

video_data = []
response = requests.get('https://api.scraperapi.com', params=params)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    videos = soup.find_all('div', {"id": "title-wrapper", "class":"style-scope ytd-video-renderer"})
    
    if videos:
        print(f"Found {len(videos)} videos.")
        temp_video_data = []
        for video in videos:
            video_details = video.find('a', {'id': 'video-title'})
            if video_details:
                title = video_details.get('title')
                link = video_details['href']
                video_url = f"https://www.youtube.com{link}"
                # Use yt_dlp to get view count
                try:
                    with YoutubeDL({'quiet': True}) as yt:
                        info = yt.extract_info(video_url, download=False)
                        views = info.get('view_count', 0)
                except Exception:
                    views = 0
                video_info = {"title": title, "link": video_url, "views": views}
                temp_video_data.append(video_info)
        # Sort by views descending
        video_data = sorted(temp_video_data, key=lambda x: x['views'], reverse=True)
        # Save the extracted data to a JSON file
        with open(f'videossearch-{search_query[0]}.json', 'w') as json_file:
            json.dump(video_data, json_file, indent=4)
    else:
        print("No videos found.")
else:
    print("Failed to retrieve the page:", response.status_code)

Found 22 videos.
